## Interacting with the simulation (real robot)
This unit will show you how to create a MoveIt Package for your industrial robot. By completing this unit, you will be able to create a package that allows your robot to perform motion planning.

### Moving the real robot
Up to now, you only moved the robot in the MoveIt application. This is very useful because you can do many tests without worrying about any damage. However, the final goal will always be to move the real robot, right?

The MoveIt package you've created is able to provide the necessary ROS services and actions in order to plan and execute trajectories, but it isn't able to pass these trajectories to the real robot. All the kinematics you've been performing were executed in an internal simulator that MoveIt provides. In order to communicate with the real robot, it will be necessary to do a couple of modifications to the MoveIt package you created in the previous chapter.

Obviously, in this course, you don't have a real robot to do this with, so you will apply the same, but for the moving, to the simulated robot. In order to see what you need to change in your MoveIt package, just follow the next exercise.	

First of all, let's have a look at the file ros_controllers.yaml that has been created by the MoveIt Setup Assistant. If you remember, this file was configured in the Setup ROS Controllers section from the previous chapter. You will find this file inside the config folder.

Let's have a look at the last part of the file, where we define the controller_list:

In [ ]:
# Simulation settings for using moveit_sim_controllers
moveit_sim_hw_interface:
  joint_model_group: arm
  joint_model_group_pose: home
# Settings for ros_control_boilerplate control loop
generic_hw_control_loop:
  loop_hz: 300
  cycle_time_error_threshold: 0.01
# Settings for ros_control hardware interface
hardware_interface:
  joints:
    - shoulder_pan_joint
    - shoulder_lift_joint
    - elbow_joint
    - wrist_1_joint
    - wrist_2_joint
    - wrist_3_joint
  sim_control_mode: 1  # 0: position, 1: velocity
# Publish all joint states
# Creates the /joint_states topic necessary in ROS
joint_state_controller:
  type: joint_state_controller/JointStateController
  publish_rate: 50
controller_list:
  - name: arm_controller
    action_ns: follow_joint_trajectory
    default: True
    type: FollowJointTrajectory
    joints:
      - shoulder_pan_joint
      - shoulder_lift_joint
      - elbow_joint
      - wrist_1_joint
      - wrist_2_joint
      - wrist_3_joint

Spawn the ur5e arm in gazebo:

In [ ]:
roslaunch ur_e_gazebo ur5e.launch limited:=true

Verify the topics related to the arm_controller:

mpuig@mpuig-vb:~/ur5arm_ws$ rostopic list 

/arm_controller/command

/arm_controller/follow_joint_trajectory/cancel

/arm_controller/follow_joint_trajectory/feedback

/arm_controller/follow_joint_trajectory/goal

/arm_controller/follow_joint_trajectory/result

/arm_controller/follow_joint_trajectory/status

/arm_controller/state


So basically, here you are defining the Action Server that you will use for controlling the joints of your robotic arm (all the joints listed below the joints section). For this specific case, you are saying that you will control the joints of your arm using an Action Server that will look like this:

/arm_controller/follow_joint_trajectory/<action_topics>

in general is:

In [ ]:
/<name>/<action_ns>/<action_topics>

But if you have to verify if the topic has the same sintaxis:

type: rostopic list | grep arm

If there is a difference, you have to change the action_ns: with the exact topic name.

Great! So now we've solved the issue related to the configuration of the Action Server, but there's still one last thing we need to do.

Basically, we are going to create a new launch file that will start all the required elements that MoveIt needs in order to be able to control our simulated robot. You can name this new file rbkairos_planning_executon.launch, and it should look like this: